# How to use `locals`
`locals` is a pure Python package that ingests JWST Wide-Field Slitless Spectroscopy data and returns a source catalog of all the low-mass stars in the field along with their calculated fundamental and secondary parameters.

### Requirements
- pip install SEDkit
- pip install bokeh

In [1]:
# Imports
from locals import source, catalog
from SEDkit import sed, spectrum, synphot
import astropy.units as q
import astropy.table as at
import numpy as np
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

## Making an SED for a single source (no WFSS spectra)

For each spectrum in our WFSS data products, a `Source` object is created. `locals` then searches Vizier to find ancillary photometry, parallaxes, and spectral types. Then it assembles an SED from the available data and calculates fundamental and atmospheric parameters.

Let's pretend one of the brown dwarfs in our field is [LHS2924](http://simbad.u-strasbg.fr/simbad/sim-id?Ident=LHS2924&NbIdent=1&Radius=2&Radius.unit=arcmin&submit=submit+id). Here is what `locals` is doing for each source under the hood.

In [2]:
# Coordinates of LHS 2924 (Usually taken from the WFSS source_list)
ra = 217.180137*q.deg
dec = 33.177539*q.deg

# Make the source object
src = source.Source(ra=ra, dec=dec, name='LHS 2924', spectral_type='L3.5')

# Find some photometry
src.find_SDSS()
src.find_2MASS()
src.find_WISE()
src.find_PanSTARRS()

# Find a distance from Gaia DR2
src.find_Gaia()

# Take a look
print('\n',src.photometry,'\n')

Setting radius to (<Quantity 0.10127179992838364 solRad>, <Quantity 0.02008219196144978 solRad>)
Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)

   band           eff         app_magnitude ...      abs_flux_unc      bandpass
                  um                       ... erg / (Angstrom cm2 s)         
-------- ------------------- ------------- ... ---------------------- --------
  SDSS.u 0.35565239687948447 21.9449996948 ... 1.2549111006613915e-18   SDSS.u
  SDSS.g  0.4702495279331733 20.9559993744 ...  8.182773370880929e-19   SDSS.g
   PS1.g 0.48491168757054565 20.5669994354 ... 1.3484427632650426e-18    PS1.g
  SDSS.r  0.6175578881059804 18.5820007324 ...  9.186342638736616e-19   SDSS.r
   PS1.r  0.6201185707505671 18.4883995056 ...   6.60097046064012e-19    PS1.r
  Gaia.G  0.6597013110557894       16.6299 ...                    0.0   Gaia.G
  SDSS.i  0.7489976846298569 16.2070007324 ...  2.544563527396815e-18   SDSS.i
   PS1.i  0.7534955578987176 16.2096004486 ...  1.99242

Now we can calculate the fundamental parameters and plot the results.

In [3]:
src.results

param,value,unc,units
object,object,object,object
name,LHS 2924,--,--
age,6.0,4.0,Gyr
distance,10.99,0.02,pc
parallax,90.9962,0.12710000574588776,mas
radius,0.10127179992838364,0.02008219196144978,solRad
SpT,L3.5,--,--
spectral_type,73.5,0.5,--
membership,--,--,--
fbol,6.66e-11,7.11e-13,erg / (cm2 s)


Let's fit a blackbody for fun.

In [4]:
src.fit_blackbody()


Blackbody fit: 1844 K


In [5]:
fig = src.plot()

## Make SEDs for all sources in the WFSS field
Now that we see the output of a single source, we can run this on the entire `SourceCatalog` generated from the WFSS output. The difference here is that a WFSS spectrum will be included in each `Source`.

In [54]:
# Path to the JWST pipeline output directory
cat_path = resource_filename('locals', 'data/fake/')

# Initialize empty source catalog
cat = catalog.SourceCatalog(cat_path)

All the sources are stored as `Source` instances within the `SourceCatalog.sources` attribute, just as the example above.

In [55]:
# Get a source instance
source1 = cat.results['SED'][0]

# Calculate the results
print(source1.results)

# Plot it
fig = source1.plot(spectra='all')

    param      value     unc        units    
------------- -------- -------- -------------
         name Source 0       --            --
          age      6.0      4.0           Gyr
     distance       --       --            --
     parallax       --       --            --
       radius       --       --            --
          SpT       --       --            --
spectral_type       --       --            --
   membership       --       --            --
         fbol 5.68e-13 1.15e-15 erg / (cm2 s)
         mbol   19.133    0.002            --
         Lbol       --       --            --
     Lbol_sun       --       --            --
         Mbol       --       --            --
         logg       --       --            --
         mass       --       --            --
         Teff       --       --            --
      Teff_bb       --       --            --


Hooray! Let's take a look at another one.

In [60]:
# Get the source instance
source2 = cat.results['SED'][1]

# Calculate the results
print(source2.results)

# Plot it
fig = source2.plot(spectra='all')

    param      value     unc        units    
------------- -------- -------- -------------
         name Source 1       --            --
          age      6.0      4.0           Gyr
     distance       --       --            --
     parallax       --       --            --
       radius       --       --            --
          SpT       --       --            --
spectral_type       --       --            --
   membership       --       --            --
         fbol 2.56e-13 4.96e-16 erg / (cm2 s)
         mbol   19.998    0.002            --
         Lbol       --       --            --
     Lbol_sun       --       --            --
         Mbol       --       --            --
         logg       --       --            --
         mass       --       --            --
         Teff       --       --            --
      Teff_bb       --       --            --


Fantastic!

Let's see a table of the results and plot all the SEDs!

In [61]:
print(cat.results)

   name    age   age_unc ...  NIRISS.F200W_unc M_NIRISS.F200W M_NIRISS.F200W_unc
           Myr     Myr   ...                                                    
--------- ------ ------- ... ----------------- -------------- ------------------
 Source 0 6000.0  4000.0 ... 0.000459071330757            0.0                0.0
 Source 1 6000.0  4000.0 ... 0.000458765862186            0.0                0.0
 Source 2 6000.0  4000.0 ... 0.000459352062721            0.0                0.0
 Source 3 6000.0  4000.0 ... 0.000454618946849            0.0                0.0
 Source 4 6000.0  4000.0 ... 0.000458916820457            0.0                0.0
 Source 5 6000.0  4000.0 ... 0.000454686655124            0.0                0.0
 Source 6 6000.0  4000.0 ... 0.000459023011072            0.0                0.0
 Source 7 6000.0  4000.0 ... 0.000459979489759            0.0                0.0
 Source 8 6000.0  4000.0 ... 0.000458739869755            0.0                0.0
 Source 9 6000.0  4000.0 ...

In [64]:
# We imposed a color cut of NIRISS.F150W-NIRISS.F200W > 0.3
# Let's see if it worked
show(cat.plot('NIRISS.F150W-NIRISS.F200W', 'fbol'))